In [3]:
import gym
import seagul.envs
import numpy as np
from numpy import pi, sin, cos
import time

env = gym.make('su_cartpole-v0')

In [4]:
def control(env, q):
    # Ausutay Ozmen
    if (q[0] < 140 * pi/180) or (q[0] > 220 * pi/180 ):
        # swing up
        # energy error: Ee
        Ee = 0.5 * env.mp * env.L * env.L * q[2] ** 2 - env.mp * env.g * env.L * (1 + cos(q[0]))
        # energy conrol gain:
        k = 0.23
    
        # input acceleration: A (of cart)
        A = k * Ee * cos(q[0]) * q[2]
        # convert A to u (using EOM)
        delta = env.mp * sin(q[0]) ** 2 + env.mc
        u = A * delta - env.mp * env.L * (q[2] ** 2) * sin(q[0]) - env.mp * env.g * sin(q[2]) * cos(q[2])
    else:
        # balancing
        # LQR: K values from MATLAB
        k1 = 140.560
        k2 = -3.162
        k3 = 41.772
        k4 = -8.314
        print("balancing")
        u = -25*(k1 * (q[0] - pi) + k2 * q[1] + k3 * q[2] + k4 * q[3])
    return u

In [6]:
obs = env.reset() 
obs,_,_,_ = env.step(np.array([.3]))
for _ in range(2000):
    actions = control(env, obs)
    obs, reward, done, _ = env.step(actions)
    #action_hist[i, :] = np.copy(actions)
    #state_hist[i, :] = np.copy(obs)
    #reward_hist[i, :] = np.copy(reward)
    #time.sleep(.1)

    env.render()
    if done:
        break

balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
balancing
